In [1]:
import pandas as pd

# Load cleaned interaction data (ground truth)
interactions = pd.read_csv("../Data/processed/interactions_cleaned.csv")

# Load trained item similarity matrix (model output)
item_similarity_df = pd.read_csv(
    "../milestone-2/item_similarity_matrix.csv",
    index_col=0
)

print("Data loaded successfully")
print("Interactions data:")
print(interactions.head())

print("\nItem similarity matrix:")
print(item_similarity_df.head())


Data loaded successfully
Interactions data:
   user_id  product_id  rating
0  user_id  product_id     NaN
1        1         101     5.0
2        1         103     3.0
3        2         102     4.0
4        3         101     4.0

Item similarity matrix:
          101  102       103       104  105
101  1.000000  0.0  0.780869  0.624695  0.0
102  0.000000  1.0  0.000000  0.000000  0.0
103  0.780869  0.0  1.000000  0.000000  0.0
104  0.624695  0.0  0.000000  1.000000  0.0
105  0.000000  0.0  0.000000  0.000000  1.0


In [2]:
def recommend_top_n(product_id, top_n=5):
    if str(product_id) not in item_similarity_df.columns:
        return []

    scores = item_similarity_df[str(product_id)].sort_values(ascending=False)
    recommended_products = scores.iloc[1:top_n+1].index.tolist()

    return recommended_products


In [3]:
recommend_top_n("101", top_n=5)


[103, 104, 102, 105]

In [6]:
def calculate_metrics(interactions, top_n=5):
    precision_list = []
    recall_list = []

    user_groups = interactions.groupby("user_id")

    for user_id, group in user_groups:
        actual_products = group["product_id"].astype(str).tolist()

        recommended_products = set()
        for product in actual_products:
            recs = recommend_top_n(product, top_n)
            if recs is not None:
                recommended_products.update(recs)

        # Skip users with no recommendations
        if len(recommended_products) == 0:
            continue

        actual_set = set(actual_products)
        recommended_set = set(recommended_products)

        true_positives = len(actual_set & recommended_set)

        if len(recommended_set) == 0 or len(actual_set) == 0:
            continue

        precision = true_positives / len(recommended_set)
        recall = true_positives / len(actual_set)

        precision_list.append(precision)
        recall_list.append(recall)

    # Final safety check
    if len(precision_list) == 0 or len(recall_list) == 0:
        return 0, 0, 0

    avg_precision = sum(precision_list) / len(precision_list)
    avg_recall = sum(recall_list) / len(recall_list)

    if avg_precision + avg_recall == 0:
        f1_score = 0
    else:
        f1_score = (2 * avg_precision * avg_recall) / (avg_precision + avg_recall)

    return avg_precision, avg_recall, f1_score


In [7]:
precision, recall, f1 = calculate_metrics(interactions, top_n=5)

print("Precision:", round(precision, 3))
print("Recall:", round(recall, 3))
print("F1-score:", round(f1, 3))


Precision: 0.0
Recall: 0.0
F1-score: 0


In [8]:
for n in [3, 5, 10]:
    precision, recall, f1 = calculate_metrics(interactions, top_n=n)
    print(f"Top-{n} Recommendations → Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}")


Top-3 Recommendations → Precision=0.000, Recall=0.000, F1=0.000
Top-5 Recommendations → Precision=0.000, Recall=0.000, F1=0.000
Top-10 Recommendations → Precision=0.000, Recall=0.000, F1=0.000
